# <center> Cálculo dos atributos de vizinhança para um espaço celular </center>

<br/>

<div style="text-align: center;font-size: 90%;">
    Bruno Dias dos Santos<sup><a href="https://orcid.org/0000-0001-6181-2158"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Gilberto Eidi Teramoto Oliveira<sup><a href="https://orcid.org/0000-0002-0082-9498"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Mestrado em Sensoriamento Remoto, Instituto Nacional de Pesquisas Espaciais (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brasil
    <br/><br/>
    Contato: <div><a href="mailto:bruno.santos@inpe.br">bruno.santos@inpe.br</a></div>
    <div><a href="mailto:gilberto.eidi@inpe.br">gilberto.terramoto@inpe.br</a></div>
    <br/><br/>
    Última atualização: 23 de Agosto de 2021
</div>
<br/>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
<b>Resumo.</b> Este Notebook apresenta uma metodologia para se calcular atributos de vizinhança em espaços celulares. Para cada célula é calculado de maneira normalizada (indo de -1 a 1), o quão maior ou quão menor é o valor dos seus atributo em comparação com as células adjacentes e com o restante do espaço celular. Para rodar esse notebook, é nessário: um arquvio .csv com os atributos do espaço celular e uma matriz de distância (.csv) informando quais são os vizinhos de cada célula. Como saída, serão retornados para cada atributo a média das células vizinhas, a diferença normalizada para as células vizinhas e a diferença normalizada em relação ao espaço celular como um todo.    
</div>    

<br/>

Importação das bibliotecas <i>Pandas</i> e <i>Numpy</i>:

In [ ]:
import pandas as pd
import numpy as np

Leitura da tabela CSV com os atributos da grade regular e criação de um Dataframe:

In [ ]:
grade = pd.read_csv("C:\\Users\\dias-bruno\\Desktop\\INPE-SER\\3PERIODO\\ANES\\20211012\\grade_maraba_com_segreg.csv", low_memory=False)

Visualização das informações da grade regular:

In [ ]:
grade

Definicação do campo identificador de cada unidade celular:

In [ ]:
id_grade = 'id'

Ordenando a grade pelos campo identificador:

In [ ]:
grade = grade.sort_values(by=[id_grade])
grade

Caso haja uma variável categórica relacionada ao _target_ (ou amostras/clases), está deverá ser excluída das métricas ser declarada e porteriormente excluída das métricas de vizinhanças:

In [ ]:
amostras = 'TARGET'

Criando um DataFrame com as informações relacionadas ao identificador da célula e, caso exista, target:

In [ ]:
var_num = grade.select_dtypes(include=['float64','int64','int','float'])
var_cat = grade.select_dtypes(include=['string','object'])

try: 
    var_cat = var_cat.drop(columns = amostras)
except:
    var_num = var_num.drop(columns = amostras)
    
var_num = var_num.fillna(0)

aux = grade[[id_grade, amostras]]

try:
    var_cat = pd.get_dummies(var_cat, drop_first=True)
    dataframe = (aux.merge(var_num, left_on=id_grade, right_on=id_grade)).merge(var_cat, left_index=True, right_index=True)
    
except:
    dataframe = (aux.merge(var_num, left_on=id_grade, right_on=id_grade))
    print("Não há variáveis categóricas para aplicar OneHotEncode")

dataframe

In [ ]:
saida = dataframe[[id_grade,amostras]]
saida

Leitura do cabeçalho do DataFrame grade. Caso haja uma variável relacionada ao <i>target</i> - ou classe que a amostra pertença -, ela será retirada do cabeçalho:

In [ ]:
dataframe.head()

In [ ]:
#rodar esta célula apenas se houver target no seu dataframe

dataframe = dataframe.drop(columns = amostras)

Leitura da matriz de distância. O processo para a geração de uma matriz de distância pode ser visualizado neste <a href = https://docs.qgis.org/2.8/en/docs/user_manual/processing_algs/qgis/vector_analysis_tools/distancematrix.html> link</a>. 

<i><b>obs:</b> Para uma grade formada por quadrados, considere 8 vizinhos. Para uma grade formada por hexágonos, considere 6 vizinhos.</i>

In [ ]:
matriz = pd.read_csv("C:\\Users\\dias-bruno\\Desktop\\INPE-SER\\3PERIODO\\ANES\\matriz_distancia_maraba.csv", low_memory=False)

Visualização da matriz. Como resultado, são apresentados a distância entre uma célula de origem e seus vizinhos adjacentes:

In [ ]:
matriz

Definicação da variável identificadora de cada unidade de origem da matriz:

In [ ]:
id_origem = 'InputID'

Definicação da variável identificadora de cada unidade de destino da matriz:

In [ ]:
id_destin = 'TargetID'

Preenchimento da matriz de distância com as informações dos vizinhos:  

In [ ]:
dados = matriz.merge(dataframe, left_on =id_destin, right_on=id_grade)
dados 

Obtendo as médias da vizinhança (MV) para cada variável e agrupando pela célula de origem:

In [ ]:
MV = (dados.groupby(id_origem, sort=True).mean())
MV.insert(0,'InputID',MV.index)
MV.index = dataframe.index

Visualização dos valores médios para cada variável agrupado pela célula de origem:

In [ ]:
MV

Passando a identificação de origem para a variável correspondente à *id_grade*:

In [ ]:
MV[id_grade] = MV[id_origem]
MV

Ajustando *mean_viz* para ter a mesma quantidade de colunas e linhas que a grade regular:

In [ ]:
MV = MV[dataframe.columns]
MV

Calculo da diferença normalizada entre cada célula e seus vizinhos adjacentes:

In [ ]:
DV = (dataframe - MV)/(dataframe + MV)

Ajuste de possíveis valores 'Nan' através de um valor _dummy_ a sua escolha:

In [ ]:
dummy = 0 #alterar para algum outro valor a sua escolha

DV = DV.fillna(dummy)

Visualização da diferença normalizada:

In [ ]:
DV

Ajuste da variável de identificação:

In [ ]:
DV[id_grade] = dataframe[id_grade]

In [ ]:
DV

Alterando a identificação das colunas da média da vizinhança (mean_viz): 

In [ ]:
MV.columns = 'MV_' + MV.columns

In [ ]:
MV

Alterando a identificação das colunas da diferença normalizada (DV): 

In [ ]:
DV.columns = 'DV_' + DV.columns

In [ ]:
DV

Calculo da diferença normalizada entre cada célula e a média geral (DG):

In [ ]:
DG = (dataframe - dataframe.mean())/(dataframe + dataframe.mean())

Visualização da diferença normalizada:

In [ ]:
DG

Ajuste da variável de identificação:

In [ ]:
DG[id_grade] = dataframe[id_grade]

In [ ]:
DG

Alterando a identificação das colunas da diferença normalizada (DG): 

In [ ]:
DG.columns = 'DG_' + DG.columns

In [ ]:
DG

Merge entre as tabelas de *MV*, *DV* e *DG*:

In [ ]:
aux = dataframe.merge(MV, left_on =id_grade, right_on='MV_'+id_grade)
aux2 = aux.merge(DV, left_on =id_grade, right_on='DV_'+id_grade)
result = aux2.merge(DG, left_on = id_grade, right_on='DG_'+id_grade)

In [ ]:
result

Incluindo a variável categórica (apenas caso esta tenha sido excluída anteriormente):

In [ ]:
result = result.merge(saida, left_on =id_grade, right_on=id_grade)
result

Exportando arquivo CSV como todos os novos atributos:

In [ ]:
result.to_csv("C:\\Users\\dias-bruno\\Desktop\\INPE-SER\\3PERIODO\\ANES\\20211012\\grade_maraba_com_segreg_viz.csv")